## Harvard Art Museums: HODP Project

In [ ]:
# all imports
import urllib3
import json
import pandas as pd
import numpy as np
http = urllib3.PoolManager()

In [ ]:
# actual thing. ~15,000? 
df = pd.DataFrame()
pg = 1
pages = 150
size = 100
httpAns = http.request('GET', 'https://api.harvardartmuseums.org/object',
fields = {
    'apikey': 'e85c3817-9b86-4fd2-8926-6b7c61c441fb',
    #'datebegin': 0,
    #'dateend': 2020,
    'size': size,
    'page': pg,
    'sort': 'random',
    'fields': 'objectnumber,title,culture,datebegin,dateend,century,period,accessionyear,totaluniquepageviews,rank,division,department,url'
})

while pg <= pages: #changed this to leq because we getting 14900 not 15000
    ans = json.loads(httpAns.data.decode('utf-8')) 
    df = df.append(ans['records'], ignore_index=True)
    httpAns = http.request('GET', ans['info']['next'])
    pg += 1
print(len(df))


15000


In [ ]:
# export data
df.to_csv('museumDataFull.csv', index = False, header=True)
# changed from museumData to museumDataFull


In [ ]:
# keywords
kw = pd.DataFrame()
kPg = 1
kPages = 70
size = 100
words = ['peace', 'love', 'war']
for word in words:
    httpAns = http.request('GET', 'https://api.harvardartmuseums.org/object',
    fields = {
        'apikey': 'e85c3817-9b86-4fd2-8926-6b7c61c441fb',
        'keyword': word,
        'size': size,
        'page': kPg,
        'sort': 'random',
        'fields': 'objectnumber,title,culture,datebegin,dateend,century,period,accessionyear,totaluniquepageviews,rank,division,department,url'
    })

    while kPg < kPages:
        ans = json.loads(httpAns.data.decode('utf-8')) 
        recs = ans['records']
        for r in recs:
            r['keyword'] = word
        kw = kw.append(recs, ignore_index=True)
        # if no more pages
        if ans['info']['pages'] == ans['info']['page']:
            break
        httpAns = http.request('GET', ans['info']['next'])
        kPg += 1

kw.to_csv('museumDataKW.csv', index = False, header=True)
print(len(kw))


2622


In [ ]:
# cultures
cDf = pd.DataFrame()
cPg = 1
cPages = 4
size = 100

httpCulture = http.request('GET', 'https://api.harvardartmuseums.org/culture',
    fields = {
         'apikey': 'e85c3817-9b86-4fd2-8926-6b7c61c441fb',
        'size': 100,
        'page': cPg
    })

while cPg < cPages:
    cAns = json.loads(httpCulture.data.decode('utf-8')) 
    cDf = cDf.append(cAns['records'], ignore_index=True)
    httpAns = http.request('GET', cAns['info']['next'])
    cPg = cPg + 1

with open("cultures.txt", "w", encoding='utf-8') as out:
    for name in cDf['name']:
        print(name, file=out)

# https://github.com/harvardartmuseums/api-docs/blob/master/sections/culture.md

